In [1]:
import gurobipy as gp
from gurobipy import *
import numpy as np
from numpy import loadtxt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopy.distance
import random
import time
from heapq import nsmallest

from scipy.stats import cauchy

In [2]:
#### Revise ######
def nearest_YSol(q_jt, c_ij, X_sol, V_1, V_2, R, t): #only R in range 0,1,2, t =0, 1, 2, 3

    y = {}
    p = {}

    idx_x = [idx for idx, element in enumerate(X_sol) if condition(element)]
    
    for i in V_1:
        
        # assign nearest depot in r = 0
        cost_can = {}
        for i_x in idx_x:
            cost_can[i_x] = c_ij[i,i_x]
        min_i_x = min(cost_can, key=cost_can.get)
        y[i, min_i_x, 0] = 1
        # assign nearest depot in r = 1
        del cost_can[min_i_x]
        back_i_x = min(cost_can, key=cost_can.get)
        y[i, back_i_x, 1] = 1
        # assign emergency in r = 2, emergency is the last node
        y[i, V_2[-1], 2] = 1
                

    
    # Sort the dictionary keys using the custom sorting function
    sorted_keys = sorted(y, key=sort_func)
    # Create a new dictionary with the sorted keys
    sorted_dict = {key: y[key] for key in sorted_keys}
    
    for i,j,r in list(sorted_dict.keys()):
        if r == 0 :
            p[i, j, 0]  = 1-q_jt[j, t]
            q_distrupt_0 = q_jt[j, t] # r=0 disrupt
        elif r == 1 :    
            p[i, j, r]  =  q_distrupt_0*(1-q_jt[j, t]) # disrupt*survive
            q_distrupt_1 = q_jt[j, t]
        else:    
            p[i, j, r]  = q_distrupt_0*q_distrupt_1 # disrupt*disrupt
            
    '''
    for i in V_1:
        for j in open_loc:       
            for r in R[1:]:
                q_sum = quicksum(q_jt[k, t]*p[i, k, r-1]*y[i, k, r-1]/(1-q_jt[k, t]) for k in open_loc[:-1]).getValue()
                p[i, j, r]  = (1-q_jt[j, t]) * q_sum
    '''
    w = {}
    for i,j,r in list(y.keys()):
        w[i,j,r] = p[i,j,r] * y[i,j,r]
        
    '''for i in V_1:
        for j in open_loc:
            for r in R:
                w[i,j,r] = p[i,j,r] * y[i,j,r]'''
    #val_cost = quicksum(p[i, j, r]*y[i,j,r]*c_ij[i, j]*d_i[i] for i,j,r in A).getValue()
    return y, w

In [4]:
def condition(x): 
    return x > 0.01

def cluster_move(x_tem, w, c_ij, q_jt, t, Hull_set, T):
    x = x_tem.copy()
    #get index candidate from x[j]
    idx_x = [idx for idx, element in enumerate(x) if condition(element)]

    #get index candidate from y[i,j,0] given j
    Arc = {} 

    for j in idx_x: #check customer each cluster and intersec with convex hull
        idx_y = []
        for i in range(len(x)):
            if (i,j,0) in list(w.keys()):
                idx_y.append(i)
                
        intersec_set = list(set(idx_y).intersection(set(Hull_set)))

        if len(intersec_set) >1:
            Arc[j] = intersec_set

    # single cluster move
    j = np.random.choice(list(Arc.keys()))
    q_i = []

    for i in Arc[j]:
        q_i.append(q_jt[i,t])
    prob_q = np.exp(np.array(q_i)/(T*0.01)*(-1))/np.sum(np.exp(np.array(q_i)/(T*0.01)*(-1))) #Boltzmann select
    i = np.random.choice(Arc[j], p = list(prob_q))
    #move j to i
    x[j] = 0
    x[i] = 1
            
    return x
    
# cross move
# Move high disrupt to low disrupt
def cross_move(x, w, q_jt, c_ij, t, T,Hull_set):
    x_tem = x.copy()
    
    ####Add in the cluster with high variable cost
    #get index candidate from x[j]
    idx_x = [idx for idx, element in enumerate(x_tem) if condition(element)]
    Arc = {} 
        
    for j in idx_x: #check customer each cluster and intersec with convex hull
        idx_y = []
        for i in range(len(x)):
            if (i,j,0) in list(w.keys()):
                idx_y.append(i)
        Arc[j] = idx_y
    
    # probability to close         
    q_j = []
    for i in list(Arc.keys()):
        q_j.append(q_jt[i,t])
    prob_qj = np.exp(np.array(q_j)/(T*0.01))/np.sum(np.exp(np.array(q_j)/(T*0.01)))                                     
    m = np.random.choice(list(Arc.keys()) , p = list(prob_qj))

    # probability to open
    add_set = list(set(Hull_set) - set(idx_x))
    if len(add_set) == 0:
        add_set.append(m)
    fill_out = np.zeros(len(x_tem))#Should not include last node
    for i in add_set:
        fill_out[i] = 1
    #Filter out by hull set
    q_i = q_jt[:-1,t] #Should not include last node
    prob_qi = np.exp(q_i*(-1)/(T*0.01)) * fill_out #Boltzmann select
    prob_qi = prob_qi / np.sum(prob_qi)
    n= np.random.choice( list(range(len(x_tem))) , p = list(prob_qi)) #Should not include last node
    
    x_tem[m] = 0
    x_tem[n] = 1
    return x_tem
#############################
#remove
# Remove high disrupt
def remove_move(x, q_jt , t, T):
    x_tem = x.copy()
    idx_x = [idx for idx, element in enumerate(x_tem) if condition(element)]
    # probability to close         
    q_j = []
    for i in idx_x:
        q_j.append(q_jt[i,t])
    prob_qj = np.exp(np.array(q_j)/(T*0.01))/np.sum(np.exp(np.array(q_j)/(T*0.01)))                                     
    m = np.random.choice(idx_x, p = list(prob_qj))
    x_tem[m] = 0
    
    return x_tem

#############################
#Add facility 
def add_move(x, q_jt , t, T, Hull_set):
    x_tem = x.copy()
    idx_x = [idx for idx, element in enumerate(x_tem) if condition(element)]
    add_set = list(set(Hull_set) - set(idx_x))
    fill_out = np.zeros(len(x_tem))#Should not include last node
    for i in add_set:
        fill_out[i] = 1
    #Fill out by hull set
    q_i = q_jt[:-1,t] #Should not include last node
    prob_qi = np.exp(q_i*(-1)/(T*0.01)) * fill_out
    prob_qi = prob_qi / np.sum(prob_qi)
    n= np.random.choice( list(range(len(x_tem))) , p = list(prob_qi)) #Should not include last node
    x_tem[n] = 1
    
    return x_tem

#############################

def balance_move(x_tem, w, c_ij, q_jt, t, Hull_set, T):
    x = x_tem.copy()
    #get index candidate from x[j]
    idx_x = [idx for idx, element in enumerate(x) if condition(element)]

    #get index candidate from y[i,j,0] given j
    Arc = {} 
    for j in idx_x: #check customer each cluster and intersec with convex hull
        idx_y = []
        
        #########################
        for i in range(len(x)):
            if (i,j,0) in list(w.keys()):
                idx_y.append(i)
        #########################        
        intersec_set = list(set(idx_y).intersection(set(Hull_set)))

        if len(intersec_set) >1:
            Arc[j] = intersec_set

            q_i = []
            for i in Arc[j]:
                q_i.append(q_jt[i,t])
            prob_q = np.exp(np.array(q_i)/(T*0.01)*(-1))/np.sum(np.exp(np.array(q_i)/(T*0.01)*(-1)))
            i = np.random.choice(Arc[j], p = list(prob_q))
            #move j to i
            x[j] = 0
            x[i] = 1

        
        
    return x
#############################

    
def cluster_move_2n_expected(x_tem, w, c_ij, d_i, q_jt, f_j ,total_cost, y, A, V_1, V_2, R,t , X_init, open_cost, Hull_set):
    best_cost = total_cost
    x = x_tem.copy()
    #get index candidate from x[j]
    idx_x = [idx for idx, element in enumerate(x) if condition(element)]

    #get index candidate from y[i,j,0] given j
    Arc = {} 
    for j in idx_x:
        idx_y = [idx for idx, element in enumerate(w[:,j, 0]) if condition(element)]
        Arc[j] = idx_y


    # We move j to i index
    # Recieve candidate cost 
    # Assump. i service by single j node
    
    val_ij = {}
    # multiple cluster move
    for j in idx_x: #loop facility
        val_ij = {}
        cost_mn ={}

        for a in list(set(Arc[j]).intersection(set(Hull_set))): # j move to a
            #level 0 estimate
            val_ij[(a,j)] = quicksum(c_ij[i, a]*d_i[i] for i in Arc[j]).getConstant()*(1-q_jt[a, t])

            #level 1, r estimate
            val_ij[(a,j)] = quicksum(w[i, b, r]*c_ij[i, b]*d_i[i]*(q_jt[a, t]/q_jt[j, t]) for i in Arc[j] for b in V_2 for r in R[1:]).getConstant() + val_ij[(a,j)]

            #fixed cost
            val_ij[(a,j)] =  f_j[a] + val_ij[(a,j)]

    
        current_j = val_ij[(j,j)] + f_j[j]
        
        
        length = min(len(val_ij),5) #searching 5 neighbors
        #sorted min to max
        for i in range(0,length): 
            m,n = sorted(val_ij, key = val_ij.get)[i]
            x_can = x.copy()
            x_can[n] = 0
            x_can[m] = 1
            #check with Main problem
            if m!= n :
                ######################################## Change the assignment Use nearest assign ################
                Y_sol, W_sol = nearest_YSol(q_jt, c_ij, x_can, V_1, V_2, R, t)
                
                Val_cost = cal_Valcost(q_jt, Y_sol, c_ij, d_i, A, V_1, V_2, R, t)
                
                can_cost = cal_Fixcost(x_can, f_j, V_2) + Val_cost + cal_Opencost(x_can, X_init, open_cost)

                ##################################################################################################
                if can_cost < best_cost:
                    cost_mn[(m,n)] = can_cost
                    best_cost = can_cost
                    y = Y_sol.copy()
                    w = W_sol.copy()
        if cost_mn != {}: #Prevent more than 1 candidate select
            m,n = sorted(cost_mn, key = cost_mn.get)[0]
            x[n] = 0
            x[m] = 1
    return x, best_cost , y , w
    


In [13]:
def cal_Fixcost(x, f_j, V_2):
    
    return quicksum(f_j[j]*x[j] for j in V_2[:-1]).getValue()

def cal_Valcost(x, q_jt, y, c_ij, d_i, V_1, V_2, R, t): #Input q_jt , t =0  
    
    open_loc = [i for i in range(len(x)) if x[i]>0]
    open_loc.append(V_2[-1])
    
    p = {}
    #for i in V_1:
    #    for j in open_loc: 
    #        for r in R:
    #            p[i,j,r] = 0
    #for i in V_1:
    #    for j in open_loc:
    #        p[i, j, 0]  = 1-q_jt[j, t]
    
    
    # Sort the dictionary keys using the custom sorting function
    sorted_keys = sorted(y, key=sort_func)
    # Create a new dictionary with the sorted keys
    sorted_dict = {key: y[key] for key in sorted_keys}
    
    for i,j,r in list(sorted_dict.keys()):
        if r == 0 :
            p[i, j, 0]  = 1-q_jt[j, t]
            q_distrupt_0 = q_jt[j, t] # r=0 disrupt
        elif r == 1 :    
            p[i, j, r]  =  q_distrupt_0*(1-q_jt[j, t]) # disrupt*survive
            q_distrupt_1 = q_jt[j, t]
        else:    
            p[i, j, r]  = q_distrupt_0*q_distrupt_1 # disrupt*disrupt
    '''
    for i in V_1:
        for j in open_loc:       
            for r in R[1:]:
                q_sum = quicksum(q_jt[k, t]*p[i, k, r-1]*y[i, k, r-1]/(1-q_jt[k, t]) for k in open_loc[:-1]).getValue()
                p[i, j, r]  = (1-q_jt[j, t]) * q_sum
    '''
    
    val_cost = quicksum(p[i, j, r]*y[i,j,r]*c_ij[i, j]*d_i[i] for i,j,r in list(y.keys())).getValue()
    return val_cost

def cal_Opencost(X, X_init, open_cost): #Single period
    Total = np.sum( np.abs(X - X_init) ) *open_cost
    return Total


def sort_func(key):
    return key[0], key[2]

In [ ]:
# Test